In [23]:
# !pip install mecab-python3

# !pip install unidic-lite
# !pip install japanize-matplotlib


In [22]:
import MeCab
import unidic
import sys
import re
import unicodedata
import matplotlib.pyplot as plt
import tqdm
import japanize_matplotlib
import os
import collections
import glob
import pandas as pd


In [2]:
# Check if MeCab works
# t = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')

t = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')

sentence = 'すもももももももものうち'
print(t.parse(sentence))

すもも	名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
うち	名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ
EOS



### Here is a translation 
- すもも	名詞 (= noun),一般 (=general),*,*,*,*,すもも,スモモ,スモモ
- も	助詞 (=particle) ,係助詞 (=binding particle),*,*,*,*,も,モ,モ
- もも	名詞 (= noun),一般 (=general),*,*,*,*,もも,モモ,モモ
- も	助詞 (=particle) ,係助詞 (=binding particle),*,*,*,*,も,モ,モ
- もも	名詞 (= noun),一般 (=general),*,*,*,*,もも,モモ,モモ
- の	助詞 (=particle) ,連体化(=attributive),*,*,*,*,の,ノ,ノ
- うち	名詞(=noun),非自立(=non-indipendent),副詞可能(adverb)=,*,*,*,うち,ウチ,ウチ

In [3]:
#create the list of meaningless words such as "that" "year" "what" etc... 
with open("stopwords.txt", encoding='UTF-8') as f:
    stopwords = f.read().split("\n")
print(stopwords)

['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ', 'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ', 'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'てん', 'とおり', 'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば', 'なに', 'など', 'なん', 'はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ', 'へん', 'ぺん', 'ほう', 'ほか', 'まさ', 'まし', 'まとも', 'まま', 'みたい', 'みつ', 'みなさん', 'みんな', 'もと', 'もの', 'もん', 'やつ', 'よう', 'よそ', 'わけ', 'わたし', '', 'ハイ', '', '', '上', '中', '下', '字', '', '', '年', '月', '日', '時', '分', '秒', '週', '火', '水', '木', '金', '土', '国', '都', '道', '府', '県', '市', '区', '町', '村', '', '', '各', '第', '方', '何', '的', '度', '文', '者', '体', '人', '他', '今', '部', '課', '係', '外', '類', '達', '気', '室', '口', '誰', '用', '界', '会', '首', '別', '話', '私', '屋', '店', '家', '場', '等', '見', '際', '観', 

In [14]:

def count_word(text, pos_tags, stopwords):
    
    words_list = []
    tagger = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')
    
    node = tagger.parseToNode(text)

    while node:
        features = node.feature.split(",")
        base = features[6]
        pos_tag = features[0]
        
        # remove "stopwords"
        if pos_tag in pos_tags and base not in stopwords:
            words_list.append((base, pos_tag))
            

        node = node.next
    count = collections.Counter(words_list)

    return count

# chart
# def draw_barh_chart(y, width):
#     fig, ax = plt.subplots(figsize=(8, 4.5))
#     ax.barh(y, width)

# #     fig.savefig("word_count.png")


# if __name__ == "__main__":
#     # read files
#     #change HERE to try other files
#     with open('F:\Wikipedia\wikiextractor-master\wikiextractor\people\people鈴木みのり (カーリング選手)', 'r', encoding='UTF-8') as files:
#         text = files.read()
    
#     # read stopwords
#     with open("stopwords.txt",encoding='UTF-8') as f:
#         stopwords = f.read().split("\n")

#     pos_tags = ["名詞", "動詞", "形容詞"] 
#     #  ["名詞", "動詞", "形容詞"]=  ["noun", "verb", "adj"]
#     count = count_word(text, pos_tags, stopwords)

# top100_words = count.most_common(100)
# top100_words
    
# top100 = pd.DataFrame(top100_words, columns = ['word_tuple', "count"]) 
# top100
   
    
    
# Draw a chart
#top_words = count.most_common(10)
#words = list(map(lambda x: x[0], top_words))[::-1]
#counts = list(map(lambda x: x[1], top_words))[::-1]
#draw_barh_chart(words, counts)

In [12]:
# top100.head()

In [11]:
# top100 = top100.assign(
#     word = lambda x: x["word_tuple"].apply(lambda wt: wt[0]),
#     pos = lambda x: x["word_tuple"].apply(lambda wt: wt[1])
# )

In [5]:
# top100.head()

In [6]:
# CHANGE PATH!!!!!!!
file_list = glob.glob("F:/Wikipedia/wikiextractor-master/wikiextractor/m_scientist/*") 

In [7]:
file_list[:5]

['F:/Wikipedia/wikiextractor-master/wikiextractor/m_scientist\\手塚治虫',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/m_scientist\\ドナルド・クヌース',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/m_scientist\\マービン・ミンスキー',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/m_scientist\\ノーム・チョムスキー',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/m_scientist\\エドガー・ダイクストラ']

In [8]:
# read stopwords
with open("stopwords.txt",encoding='UTF-8') as f:
    stopwords = f.read().split("\n")

pos_tags = ["名詞", "動詞", "形容詞"] 
#  ["名詞", "動詞", "形容詞"]=  ["noun", "verb", "adj"]

In [9]:
dfs = []

In [15]:
for f in file_list:
    with open(f, 'r', encoding='UTF-8') as file:
        text = file.read()
    
    f_name = f.split("\\")[-1]
    
    count = count_word(text, pos_tags, stopwords)
    
    #top100_words = count.most_common(100)
    all_words = count.items()
    
    all_words_df = (
        pd
        .DataFrame(all_words, columns = ['word_tuple', "count"])
        .assign(
            word = lambda x: x["word_tuple"].apply(lambda wt: wt[0]),
            pos = lambda x: x["word_tuple"].apply(lambda wt: wt[1]),
            file_name = f_name
        )
        .sort_values("count", ascending=False)
    )
    #CHANGE PATH!!!!!!!
    full_csv_file = "F:/Wikipedia/wikiextractor-master/wikiextractor/m_scientist/new_csvs/" + f_name + "_words.csv"
    #print(f_name)
    all_words_df.to_csv(full_csv_file, index=None)
    dfs.append(all_words_df)

PermissionError: [Errno 13] Permission denied: 'F:/Wikipedia/wikiextractor-master/wikiextractor/m_scientist\\csvs'

In [17]:
overall = (
    pd
    .concat(dfs)
    .groupby(["word", "pos"])
    .pipe(
        lambda grp: pd.DataFrame({
            "total": grp["count"].sum(),
            "articles": grp["file_name"].nunique()
        }))
    .reset_index()
    .sort_values("total", ascending=False)
)

In [18]:
overall[overall.pos == "名詞"].head(60)

,word,pos,total,articles
46938,研究,名詞,62473,17011
34981,教授,名詞,49488,18099
26671,学,名詞,46580,14842
25240,大学,名詞,45604,15050
35852,日本,名詞,40098,16587
26735,学者,名詞,20988,14110
20936,博士,名詞,20821,12783
47806,科学,名詞,18048,7656
47802,科,名詞,15778,8436
26678,学会,名詞,14628,6171


In [19]:
overall[overall.pos == "動詞"].head(60)

,word,pos,total,articles
19942,務める,動詞,13269,7981
53265,行う,動詞,12212,6180
44757,生まれる,動詞,7858,7114
26675,学ぶ,動詞,7117,4944
49509,経る,動詞,7072,5738
46667,知る,動詞,6976,4762
21579,受ける,動詞,6603,3919
2479,できる,動詞,5433,2481
31584,得る,動詞,4571,2953
34018,持つ,動詞,4045,2501


In [20]:
overall.to_csv("m_scientist_text_overall.csv", index= False)